In [1]:
!pip install pyLDAvis==3.2.2

In [2]:
import numpy as np
import pandas as pd
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim as gensimvis

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

C:\Users\sthan\anaconda3\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [3]:
def load_data(file):
    with open(file, 'r', encoding="utf-8") as f:
        data =json.load(f)
    return (data)

def write_data(file):
    with open(file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [4]:
stopwords = stopwords.words("english")

In [5]:
#data = load_data(r"C:\Users\sthan\Downloads\Keywords categorization_For Elaine (1).json")

In [6]:
df = pd.read_csv(r"C:\Users\sthan\OneDrive\Desktop\Text Mining\EL.csv")

In [7]:
abslist = df['abstract'].tolist()

In [8]:
print(abslist[0][0:90])

The use of textual analysis methods in the accounting profession has grown markedly over t


In [9]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(abslist)
print (lemmatized_texts[0][0:90])

use textual analysis method accounting profession grow markedly past decade accounting pro


In [10]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['use', 'textual', 'analysis', 'method', 'accounting', 'profession', 'grow', 'markedly', 'past', 'decade', 'accounting', 'professional', 'as', 'well', 'business', 'accounting', 'accreditor', 'call', 'accounting', 'student']


In [11]:
#Bigrams and Trigrams
bigrams_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigrams_phrases[data_words], threshold=50)

bigram = gensim.models.phrases.Phraser(bigrams_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
  return([bigram[doc] for doc in texts])

def make_trigram(texts):
  return([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigram(data_bigrams)

print (data_bigrams_trigrams[0])


['use', 'textual', 'analysis', 'method', 'accounting', 'profession', 'grow', 'markedly', 'past', 'decade', 'accounting', 'professional', 'as_well', 'business', 'accounting', 'accreditor', 'call', 'accounting', 'student', 'acquire', 'increase', 'depth', 'breadth', 'knowledge', 'digital', 'datum', 'analytic', 'case', 'enable', 'accounting', 'instructor', 'previous', 'background', 'experience', 'textual', 'analysis', 'introduce', 'student', 'use', 'textual', 'analysis', 'accounting', 'allow', 'student', 'conduct', 'simple', 'analysis', 'use', 'freely', 'available', 'software', 'document', 'retrieve', 'publicly', 'available', 'filing', 'case', 'design', 'auditing', 'accounting', 'information', 'system', 'fraud', 'examination', 'financial_statement', 'analysis', 'course', 'use', 'accounting', 'course', 'content', 'relevant', 'document', 'subject', 'examination', 'right_reserve']


In [12]:
#TF-IDF REMOVAL

from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]

print (corpus [0][0:20])

tfidf = TfidfModel (corpus, id2word=id2word)

low_value = 0.03

words = []

words_missing_in_tfidf = []

for i in range(0, len (corpus)):
  bow = corpus[i]
  low_value_words = [] #reinitialize to be safe. You can skip this.
  tfidf_ids = [id for id, value in tfidf[bow]]
  bow_ids = [id for id, value in bow]
  low_value_words = [id for id, value in tfidf[bow] if value < low_value]
  drops = low_value_words+words_missing_in_tfidf
  for item in drops:
    words.append(id2word[item])
  words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre o will be missing

  new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
  corpus [i] = new_bow


[(0, 8), (1, 1), (2, 1), (3, 1), (4, 5), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1)]


In [13]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 8), (1, 1), (2, 1), (3, 1), (4, 5), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1)]
accounting


In [14]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [15]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=15)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.040513 -0.287806       1        1  22.202977
14     0.071126 -0.263832       2        1  14.868182
5     -0.284252 -0.168199       3        1  14.142400
13     0.218430 -0.123196       4        1  14.004259
10     0.182356 -0.221404       5        1  12.623699
4      0.270508  0.048531       6        1   7.090727
1      0.212227  0.250195       7        1   6.083711
7     -0.089294 -0.158183       8        1   4.196470
11    -0.223454  0.083603       9        1   1.629153
8     -0.008728  0.043061      10        1   0.744683
2     -0.070674  0.220103      11        1   0.691534
0     -0.121156  0.162953      12        1   0.654392
3      0.000137  0.178334      13        1   0.546403
9     -0.073188  0.102511      14        1   0.404090
12    -0.043523  0.133331      15        1   0.117320, topic_info=            Term        Freq       Total Category  logprob  loglift
145   government  422.000000  422.000000  Default  15.0000  15.0000
399   technology  121.000000  121.000000  Default  14.0000  14.0000
461         city  106.000000  106.000000  Default  13.0000  13.0000
407      citizen   80.000000   80.000000  Default  12.0000  12.0000
17         datum  153.000000  153.000000  Default  11.0000  11.0000
...          ...         ...         ...      ...      ...      ...
1771    rhetoric    0.006994    0.752199  Topic15  -8.1666   2.0701
513        clear    0.092683   12.039011  Topic15  -5.5825   1.8813
362       appear    0.092931   13.743150  Topic15  -5.5799   1.7516
754        close    0.024439    8.350005  Topic15  -6.9155   0.9142
756       divide    0.015729   16.279743  Topic15  -7.3562  -0.1942

[437 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
192       4  0.904928        academic
192       5  0.106462        academic
1401      5  0.830133       accompany
616       1  0.928003  accountability
827       4  0.868789     acquisition
...     ...       ...             ...
55        8  0.046168            well
55       10  0.011542            well
2541     11  0.928706            wepe
2716      6  0.965675           woman
1528      1  0.941871        workshop

[875 rows x 3 columns], R=15, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 15, 6, 14, 11, 5, 2, 8, 12, 9, 3, 1, 4, 10, 13])

In [16]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=lemmatized_texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=lemmatized_texts)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,10,0.9877,"government, information, initiative, organizat...",use textual analysis method accounting profess...
1,1,7,0.7619,"citizen, service, project, government, mobile,...",natural language processing part artificial in...
2,2,0,0.6655,"statement, rather, financial, language, gaap, ...",passage necessitate government agency provide ...
3,3,4,0.8082,"risk, sample, woman, information, size, includ...",project announce aim simplify user access auth...
4,4,14,0.9877,"government, information, portal, use, public, ...",paper present prototype system support tempora...
5,5,1,0.9933,"protein, cell, site, cost, population, gene, s...",critical assessment intestinal immune response...
6,6,10,0.9879,"government, information, initiative, organizat...",inter - agency collaboration increasingly impo...
7,7,4,0.5774,"risk, sample, woman, information, size, includ...",give information play decisive role emergency ...
8,8,14,0.6074,"government, information, portal, use, public, ...",city world attempt become smart use datum tech...
9,9,6,0.5556,"government, datum, use, open, model, public, p...",artificial intelligence become important tool ...


In [17]:
# Top 10 keywords from each topic

# Define the number of keywords to show for each topic
num_top_words = 10

# Get the topic keywords and store them in a list
topic_keywords = []
for topic_idx, topic in enumerate(lda_model.print_topics(num_topics=10, num_words=num_top_words)):
    topic_keywords.append([word.split("*")[1].strip() for word in topic[1].split("+")])

# Create a dataframe to display the topic keywords
keywords_df = pd.DataFrame(topic_keywords, columns=["Keyword_" + str(i) for i in range(num_top_words)], index=["Topic_" + str(i) for i in range(10)])

# Display the dataframe
print(keywords_df)

                Keyword_0      Keyword_1        Keyword_2         Keyword_3  \
Topic_0             "ict"    "inclusion"     "investment"        "regional"   
Topic_1          "neuron"     "disaster"  "vulnerability"         "release"   
Topic_2       "statement"       "rather"      "financial"        "language"   
Topic_3  "transformation"    "evolution"         "theory"    "metropolitan"   
Topic_4             "day"  "atmospheric"         "season"            "wepe"   
Topic_5        "increase"        "study"         "effect"             "use"   
Topic_6      "government"  "information"     "initiative"  "organizational"   
Topic_7            "city"        "smart"          "study"         "digital"   
Topic_8      "government"  "information"         "portal"             "use"   
Topic_9      "government"        "datum"            "use"            "open"   

               Keyword_4      Keyword_5      Keyword_6      Keyword_7  \
Topic_0           "keep"       "appear"  "territorial"   

In [18]:
# Create an empty dictionary to store the matching keywords and authors for each author
author_matches = {}

# Iterate through each row in the dataframe
for index, row in df.iterrows():
    # Get the keywords and author for the current row
    keywords = row['author.keywords'].split(',')
    author = row['Authors']
    
    # Iterate through each keyword in the row
    for keyword in keywords:
        # Check if the keyword exists in the dictionary
        if keyword.strip() in author_matches:
            # If the keyword exists, append the author to the list
            author_matches[keyword.strip()].append(author)
        else:
            # If the keyword doesn't exist, create a new list with the author
            author_matches[keyword.strip()] = [author]

# Iterate through each author and list of matching keywords
for author in df['Authors'].unique():
    # Create empty lists to store the matching keywords and authors for the current author
    matching_keywords = []
    matching_authors = []
    
    # Iterate through each keyword in the author's keywords list
    for keyword in df.loc[df['Authors'] == author, 'author.keywords'].str.split(','):
        # Strip whitespace from the keyword
        keyword = keyword[0].strip()
        
        # If the keyword exists in the author_matches dictionary, append it and its matching authors to the lists
        if keyword in author_matches:
            matching_keywords.append(keyword)
            matching_authors.extend(author_matches[keyword])
    
    # Remove duplicates from the matching authors list
    matching_authors = list(set(matching_authors))
    
    # Print the matching keywords and authors for the current author
    print(f"matching keywords: {', '.join(matching_keywords)}")
    print(f"Matching authors: {', '.join(matching_authors)}")


matching keywords: data analytics; fraud examination; text mining; textual analysis
Matching authors: Fisher I.E., Hughes M.E., Janvrin D.J.
matching keywords: computational linguistics; natural language processing; text mining
Matching authors: Fisher I.E., Garnsey M.R., Hughes M.E.
matching keywords: Accounting semantics; Formal model; XBRL
Matching authors: Fisher I.E., Nehmer R.A.
matching keywords: Accounting language; Accounting Standards Codification; FASB; Natural language processing
Matching authors: Garnsey M.R., Zelka S.L., Fisher I.E.
matching keywords: Financial accounting standards; Standards codification; Temporal reconstruction
Matching authors: Fisher I.E.
matching keywords: Feline; IEL; LPL; Peyer's patches; Phenotype; Small intestine
Matching authors: Howard K.E., Fisher I.L., Dean G.A., Burkhard M.J.
matching keywords: Comparative case study; Digital transformation; Governance of collaborations; Inter-agency collaboration; Intergovernmental collaboration
Matching au